In [1]:
import torch
import os
import json
import numpy as np
import pickle

In [13]:
parent_path = "../neurips/"
os.getcwd()
os.listdir(parent_path + "abstractbeam/models")

['model-best-valid.ckpt', 'logs', 'model-latest.ckpt']

In [136]:
for file in os.listdir(parent_path + f"abstractbeam/data/"):
    with open(parent_path + f"abstractbeam/data/" + file, "rb") as f:
        train_data = pickle.load(f)
        filter = [ele.solution.expression() for ele in train_data if "fn_14" in ele.solution.expression()]
        if len(filter) > 1:
            t = 1/0

In [61]:
with open(parent_path + "abstractbeam/models/model-best-valid.ckpt", "rb") as f:
    ckpt = torch.load(f)  # , map_location=torch.device("mps"))

In [62]:
for key in ckpt.keys():
    print(key)

step
model
optimizer
inventions
higher_order_functions
base_function_dict
dc_abstractions
domain
frontiers


In [73]:
ckpt["higher_order_functions"]

{'Map': [1, 0],
 'Filter': [1, 0],
 'Count': [1, 0],
 'ZipWith': [2, 0, 0],
 'Scanl1': [2, 0],
 'fn_14': [2, 0, 0],
 'fn_23': [1, 0]}

In [139]:
inventions = ckpt["inventions"]
for inv in inventions:
    print(inv.name, inv.program, inv.arity)

fn_2 Multiply(Subtract(x2, x1), x2) 2
fn_4 Filter((lambda v1: IsOdd(v1)), x1) 1
fn_5 Take(-1, x1) 1
fn_6 Add(1, x1) 1
fn_8 Drop(Multiply(x1, -1), x2) 2
fn_14 Access(x2(-1, x3), x1) 3
fn_22 IsEven(Min(x2, fn_6(x1))) 2
fn_23 x1(Filter((lambda v1: IsOdd(v1)), x2)) 2
fn_29 Map((lambda v1: fn_2(x1, v1)), x2) 2
fn_36 Add(x1, -1) 1


In [138]:
inventions[1]

crossbeam.abstraction.dreamcoder2lambdabeam.Invented

In [65]:
constants = ckpt["domain"].constants
print(f"initial constants: {[-1, 1]}")
print(f"now: {constants}")

initial constants: [-1, 1]
now: [-1, 1, 0]


In [66]:
with open(parent_path + "abstractbeam/results/run_1.json", "rb") as json_file:
    results = json.load(json_file)
solutions = [ele for ele in results["results"] if ele["success"]]
len(solutions) / len(results["results"]) * 100, len(ckpt['frontiers']) / len(results["results"]) * 100

(37.5, 48.75)

In [67]:
solutions

[{'elapsed_time': 0.4518822869285941,
  'num_unique_values': 216,
  'solution': 'Maximum(Filter((lambda u1: ((lambda v1: IsOdd(v1)))(u1)), x))',
  'solution_weight': 6,
  'stats': {'explored_percent_concrete': 49.376114081996434,
   'explored_percent_lambda': 28.520499108734402,
   'explored_percent_none': 22.10338680926916,
   'kept_percent_concrete': 53.55450236966825,
   'kept_percent_lambda': 46.44549763033175,
   'num_examples': 5,
   'num_explored_concrete': 277,
   'num_explored_lambda': 160,
   'num_explored_none': 124,
   'num_inputs': 1,
   'num_kept_concrete': 113,
   'num_kept_lambda': 98,
   'num_values_explored': 561,
   'num_values_kept': 211,
   'temperature': 1.0,
   'total_num_restarts': 1,
   'total_num_values_explored': 561},
  'success': True,
  'task': "Task(\n    name='filter:max_odd',\n    inputs_dict={'x': [[32, 57, 60, 53], [7, 12, 6, 18, 13, 14, 5, 6], [-6, 8, 5, -7, -13, 1, -4, 18, 32], [32, 25, 68, 43, 42, 44, 56, 37, 28, 29], [32, 25, 68, 43, 42, 44, 55, 3

In [72]:
programs = [ele["solution"] for ele in solutions if "fn" in ele["solution"]]
programs

['Sort(fn_29(fn_6(-1), x))',
 'Multiply(k, fn_6(IntDivide(x, k)))',
 'fn_29(1, x)',
 'Count((lambda u1: ((lambda v1: IsEven(((lambda v1: Min(v1, fn_6(1))))(v1))))(u1)), x)',
 'Scanl1((lambda u1, u2: ((lambda v1: fn_6(v1)))(u1)), Map((lambda u1: fn_6(-1)), x))',
 'Map((lambda u1: ((lambda v1: If(((lambda v1: Greater(v1, y)))(v1), 1, fn_6(-1))))(u1)), x)',
 'fn_29(Add(-1, -1), x)']

In [89]:
[ele["solution"] for ele in solutions if "fn_6(1)" in ele["solution"]]

['Count((lambda u1: ((lambda v1: IsEven(((lambda v1: Min(v1, fn_6(1))))(v1))))(u1)), x)']

In [74]:
categories = {}
for result in solutions:
    category = result['task'].split("name=")[1].split(":")[0]
    if category[1:] in categories.keys():
        categories[category[1:]] += 1
    else:
        categories[category[1:]] = 1

In [75]:
categories

{'filter': 4,
 'map': 11,
 'none': 8,
 'zipwith': 1,
 'scanl1': 1,
 'count': 4,
 'multi': 1}

In [76]:
t = [ele[0][0] for ele in ckpt['frontiers'].values() if  " 0" in ele[0][0] or " -1" in ele[0][0] or " 1"]

In [77]:
sols = [ele["solution"] for ele in solutions]
res = [ele[0][0] for ele in ckpt["frontiers"].values()]


In [78]:
solution_programs = [(ele["solution"], ele["task_solution"]) for ele in solutions]

In [79]:
s = [ele["task"].split("solution=")[1][1:-3] for ele in results["results"]]

In [80]:
print("Number of programs containing a constant value")
for i in range(-1, 5):
    filtered = [ele for ele in s if f" {i}" in ele]
    print(f"{i}: {len(filtered)}")

Number of programs containing a constant value
-1: 2
0: 13
1: 6
2: 7
3: 2
4: 1


In [64]:
print("Average length of programs containing a constant value")
for i in range(-1, 5):
    filtered = [len(ele.split(" ")) for ele in s if f" {i}" in ele]
    print(f"{i}: {np.mean(filtered)}")

Average length of programs containing a constant value
-1: 9.5
0: 7.6923076923076925
1: 6.333333333333333
2: 6.857142857142857
3: 7.0
4: 6.0


In [65]:
for i in range(-1, 5):
    filtered = [ele for ele in s if f" {i}" in ele]
    print(f"{i}: {filtered}")

-1: ['Filter((lambda u1: Greater(u1, -1)), ZipWith((lambda u1, u2: Subtract(u1, u2)), x, y))', 'Map((lambda u1: Multiply(If(Less(u1, 0), -1, 1), Square(u1))), x)']
0: ['Scanl1((lambda u1, u2: If(Greater(Multiply(u1, u2), 0), Subtract(0, u2), u2)), x)', 'Sum(ZipWith((lambda u1, u2: Max(Subtract(u1, u2), 0)), Reverse(s), s))', 'Count((lambda u1: Less(u1, 0)), x)', 'Filter((lambda u1: IsOdd(Min(u1, 0))), x)', 'Add(Count((lambda u1: Equal(u1, 0)), x), Count((lambda u1: Equal(u1, 0)), y))', 'Count((lambda u1: IsEven(Min(u1, 0))), x)', 'Map((lambda u1: If(Less(u1, 0), y, u1)), x)', 'Scanl1((lambda u1, u2: Add(u1, 1)), Map((lambda u1: 0), x))', 'Take(Count((lambda u1: Greater(u1, 0)), r), Sort(c))', 'Map((lambda u1: If(Greater(u1, y), 1, 0)), x)', 'Map((lambda u1: Multiply(If(Less(u1, 0), -1, 1), Square(u1))), x)', 'Scanl1((lambda u1, u2: If(Equal(u2, 0), 0, Add(u1, u2))), x)', 'Count((lambda u1: Greater(u1, 0)), ZipWith((lambda u1, u2: Subtract(u1, u2)), a, b))']
1: ['Sum(Map((lambda u1: 1),